<a href="https://colab.research.google.com/github/sayanbanerjee32/NLP-with-fastai2.0/blob/main/fastai_on_10kGNAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#!pip install fastai==2.1.4
#!pip install bpemb
!pip install cleantext 

In [8]:
import csv

from bpemb import BPEmb
from cleantext import clean
#from fastai.callbacks import *
#from fastai.imports import torch
from fastai.text import * 
import pandas as pd

#torch.cuda.set_device(2)

In [9]:
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

# construct the vocabulary by added a padding token with the ID 25000 (because of the bpemb_de vocab size)
itos = dict(enumerate(bpemb_de.words + ['xxpad']))
voc = Vocab(itos)

downloading https://nlp.h-its.org/bpemb/de/de.wiki.bpe.vs25000.model


100%|██████████| 664539/664539 [00:00<00:00, 1309323.45B/s]


downloading https://nlp.h-its.org/bpemb/de/de.wiki.bpe.vs25000.d300.w2v.bin.tar.gz


100%|██████████| 28012481/28012481 [00:02<00:00, 13082667.68B/s]


NameError: ignored